In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.metrics import classification_report

# Data Augmentation Parameters
batchsize = 64  # Increase batch size for faster training
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2,
                                   height_shift_range=0.2, validation_split=0.2)

# Data Generators
train_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                               target_size=(224, 224), batch_size=batchsize,
                                               class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                                    target_size=(224, 224), batch_size=batchsize,
                                                    class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\test',
                                             target_size=(224, 224), batch_size=batchsize,
                                             class_mode='categorical')

# Base model
base_model = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Add custom head to the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Combined model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Callbacks
checkpoint_path = r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\vgg19_model_checkpoint.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)

callbacks = [checkpoint, earlystop, learning_rate]

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data,
                    steps_per_epoch=train_data.samples // batchsize,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples // batchsize,
                    callbacks=callbacks,
                    epochs=30)

# Load the best model
model.load_weights(checkpoint_path)

# Evaluate the model on the test data
loss_test, acc_test = model.evaluate(test_data)

# Print the test accuracy and loss
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)

# Get predictions for the test data
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Map class indices to class labels
class_labels = list(test_data.class_indices.keys())

# Get true labels for the test data
true_classes = test_data.classes

# Generate classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the classification report
print(report)


Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
Epoch 1/30


C:\Users\rajat\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.7047 - loss: 0.6633
Epoch 1: val_loss improved from inf to 0.58483, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\vgg19_model_checkpoint.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 8174s 10s/step - accuracy: 0.7048 - loss: 0.6631 - val_accuracy: 0.7678 - val_loss: 0.5848 - learning_rate: 0.0010
Epoch 2/30


C:\Users\rajat\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss improved from 0.58483 to 0.49746, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\vgg19_model_checkpoint.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7333 - val_loss: 0.4975 - learning_rate: 0.0010
Epoch 3/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8427 - loss: 0.3663
Epoch 3: val_loss did not improve from 0.49746
849/849 ━━━━━━━━━━━━━━━━━━━━ 7890s 9s/step - accuracy: 0.8427 - loss: 0.3663 - val_accuracy: 0.7922 - val_loss: 0.5480 - learning_rate: 0.0010
Epoch 4/30

Epoch 4: val_loss did not improve from 0.49746
849/849 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 1.1202 - learning_rate: 0.0010
Epoch 5/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8679 - loss: 0.3187
Epoch 5: val_loss did not improve from 0.49746

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
849/849 ━